In [79]:
import numpy as np
import pandas as pd

In [80]:
all_train_X = np.load("../train_data/X_train_kaggle.npy")

In [81]:
all_train_X.shape

(1703, 10, 128)

In [82]:
# if the values in the data are in the same order as mentioned in competition info...
train_X_by_attribute = {
    "Orientation_X" : all_train_X[:,0,:],
    "Orientation_Y" : all_train_X[:,1,:],
    "Orientation_Z" : all_train_X[:,2,:],
    "Orientation_W" : all_train_X[:,3,:],
    "AngularVelocity_X" : all_train_X[:,4,:],
    "AngularVelocity_Y" : all_train_X[:,5,:],
    "AngularVelocity_Z" : all_train_X[:,6,:],
    "LinearAcceleration_X" : all_train_X[:,7,:],
    "LinearAcceleration_Y" : all_train_X[:,8,:],
    "LinearAcceleration_Z" : all_train_X[:,9,:],
}
linZ = train_X_by_attribute.get("LinearAcceleration_Z")
linZ.shape

(1703, 128)

In [110]:
# normalisointi välille 0-1. 
# 1. Aloitetaan etsimällä kaikkien attribuuttiarvojen min ja max
att_num = 10
case_num = 1703
measure_num = 128
minmax_all = []

for a in range(att_num):
    minmax_this = [np.min(all_train_X[:,a,:]), np.max(all_train_X[:,a,:])] # [minvalue, maxvalue]
    minmax_all.append(minmax_this)
    
minmax_all

[[-0.98862000000000005, 0.98909999999999998],
 [-0.98965000000000003, 0.98892999999999998],
 [-0.16283, 0.15570999999999999],
 [-0.15345, 0.15395],
 [-1.0547, 1.2116],
 [-0.92786000000000002, 1.0790999999999999],
 [-1.2444999999999999, 1.1256999999999999],
 [-29.895, 24.945],
 [-54.591000000000001, 32.622999999999998],
 [-49.402999999999999, 22.335999999999999]]

In [111]:
from IPython.display import Math
display(Math(r'z_i=\frac{x_i-\min(x)}{\max(x)-\min(x)}'))
print("missä")
display(Math(r'x=(x_1,...,x_n)'), )
print("zi normalisoitu arvo xi")
# muutetaan allaolevalla kaavalla kaikki arvot z_i välille 0-1

<IPython.core.display.Math object>

missä


<IPython.core.display.Math object>

zi normalisoitu arvo xi


In [115]:
# tässä tuo ylempi.
all_trainX_normalized = np.empty([case_num, att_num, measure_num])
for c in range(case_num):
    for a in range(att_num):
        for m in range(measure_num):
            xi = all_train_X[c,a,m]
            zi = (xi - minmax_all[a][0]) / (minmax_all[a][1] - minmax_all[a][0])
            all_trainX_normalized[c,a,m] = zi

In [117]:
# katsotaan että kaikkien attribuuttien kaikki arvot ovat välillä 0..1
print(all_trainX_normalized.shape)
for a in range(att_num):
    print(np.min(all_trainX_normalized[:,a,:]), np.max(all_trainX_normalized[:,a,:]))

(1703, 10, 128)
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [118]:
# (1) X_mean_var will have 10 lists, each holding values of one attribute -- dimension 1
# (2) all lists of (1) will hold 1703 cases -- dimension 2
# (3) all lists of (2) will hold a list of two values [mean, variance] -- "dimension 3"

# [10]         [1703]   [2]
# attributes   cases    values -> [mean, variance]

X_train_mean_var_norm = []

for a in range(att_num):
    this_attribute = []
    cur = all_trainX_normalized[:,a,:]
    
    for c in range(case_num):
        this_mean = np.mean(cur[c], axis=0)
        this_variance = np.var(cur[c], axis=0)
        this_case = [this_mean, this_variance]
        this_attribute.append(this_case)
        
    X_train_mean_var_norm.append(this_attribute)

In [119]:
# mean and standard deviation here
X_train_mean_std_norm = []

for a in range(att_num):
    this_attribute = []
    cur = all_trainX_normalized[:,a,:]
    
    for c in range(case_num):
        this_mean = np.mean(cur[c], axis=0)
        this_std = np.std(cur[c], axis=0)
        this_case = [this_mean, this_std]
        this_attribute.append(this_case)
        
    X_train_mean_std_norm.append(this_attribute)

In [8]:
df_train_y = pd.read_csv("../train_data/y_train_final_kaggle.csv")
#df_train_y.head(6)

In [120]:
df_groups = pd.read_csv("../train_data/groups.csv")
#df_groups

In [10]:
surfaces = df_train_y.Surface.unique()
surfaces

array(['soft_tiles', 'hard_tiles', 'carpet', 'fine_concrete', 'soft_pvc',
       'tiled', 'concrete', 'wood', 'hard_tiles_large_space'], dtype=object)

In [121]:
# create a pandas df for all the attributes separately. One df from one attribute and then later combine them as desired
# for testing different attribute combinations
dfs_mean_std_norm = []

# "Orientation_X"
df_Xtrain_mean_std_orientationX = pd.DataFrame(X_train_mean_std_norm[0], columns = ['OrientationX_mean', 'OrientationX_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_orientationX)

# "Orientation_Y"
df_Xtrain_mean_std_orientationY = pd.DataFrame(X_train_mean_std_norm[1], columns = ['OrientationY_mean', 'OrientationY_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_orientationY)

# "Orientation_Z"
df_Xtrain_mean_std_orientationZ = pd.DataFrame(X_train_mean_std_norm[2], columns = ['OrientationZ_mean', 'OrientationZ_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_orientationZ)

# "Orientation_W"
df_Xtrain_mean_std_orientationW = pd.DataFrame(X_train_mean_std_norm[3], columns = ['OrientationW_mean', 'OrientationW_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_orientationW)

# "AngularVelocity_X"
df_Xtrain_mean_std_angularVelocityX = pd.DataFrame(X_train_mean_std_norm[4], columns = ['AngularVelocityX_mean', 'AngularVelocityX_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_angularVelocityX)

# "AngularVelocity_Y"
df_Xtrain_mean_std_angularVelocityY = pd.DataFrame(X_train_mean_std_norm[5], columns = ['AngularVelocityY_mean', 'AngularVelocityY_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_angularVelocityY)

# "AngularVelocity_Z"
df_Xtrain_mean_std_angularVelocityZ = pd.DataFrame(X_train_mean_std_norm[6], columns = ['AngularVelocityZ_mean', 'AngularVelocityZ_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_angularVelocityZ)

# "LinearAcceleration_X"
df_Xtrain_mean_std_linearAccelerationX = pd.DataFrame(X_train_mean_std_norm[7], columns = ['LinearAccelerationX_mean', 'LinearAccelerationX_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_linearAccelerationX)

# "LinearAcceleration_Y"
df_Xtrain_mean_std_linearAccelerationY = pd.DataFrame(X_train_mean_std_norm[8], columns = ['LinearAccelerationY_mean', 'LinearAccelerationY_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_linearAccelerationY)

# "LinearAcceleration_Z"
df_Xtrain_mean_std_linearAccelerationZ = pd.DataFrame(X_train_mean_std_norm[9], columns = ['LinearAccelerationZ_mean', 'LinearAccelerationZ_std',])
dfs_mean_std_norm.append(df_Xtrain_mean_std_linearAccelerationZ)


In [123]:
df_Xtrain_mean_std_norm = pd.concat(dfs_mean_std_norm, axis=1, sort=False)
df_groups_Xtrain_mean_std_norm = pd.concat([df_groups, df_Xtrain_mean_std_norm], axis=1, sort=False)
df_groups_Xtrain_mean_std_norm

,# Id,Group Id,Surface,OrientationX_mean,OrientationX_std,OrientationY_mean,OrientationY_std,OrientationZ_mean,OrientationZ_std,OrientationW_mean,...,AngularVelocityY_mean,AngularVelocityY_std,AngularVelocityZ_mean,AngularVelocityZ_std,LinearAccelerationX_mean,LinearAccelerationX_std,LinearAccelerationY_mean,LinearAccelerationY_std,LinearAccelerationZ_mean,LinearAccelerationZ_std
0,0,13,soft_tiles,0.116272,0.000183,0.179746,0.000237,0.180059,0.001350,0.152831,...,0.460674,0.015656,0.528239,0.007198,0.549935,0.016550,0.660160,0.015850,0.558728,0.015204
1,1,31,hard_tiles,0.015176,0.000076,0.622425,0.000251,0.610537,0.001589,0.021387,...,0.458456,0.023090,0.527680,0.028381,0.547354,0.020088,0.657683,0.021294,0.557773,0.029478
2,2,20,hard_tiles,0.240966,0.000694,0.072516,0.000395,0.105040,0.001690,0.267951,...,0.457721,0.017634,0.536870,0.009189,0.547861,0.013734,0.659043,0.020213,0.557677,0.015832
3,3,31,hard_tiles,0.025004,0.000114,0.656931,0.000228,0.633468,0.001405,0.036211,...,0.460924,0.023178,0.528389,0.013560,0.548811,0.026737,0.656419,0.047985,0.556905,0.048299
4,4,22,carpet,0.049208,0.001488,0.716572,0.003104,0.699713,0.003081,0.055269,...,0.485295,0.007998,0.464988,0.016175,0.543560,0.007950,0.660129,0.012995,0.558316,0.011283
5,5,1,fine_concrete,0.734853,0.000159,0.059211,0.000113,0.111652,0.002056,0.763375,...,0.462254,0.027311,0.522158,0.022111,0.547136,0.028058,0.656545,0.026042,0.559218,0.031671
6,6,34,soft_pvc,0.296434,0.001068,0.043723,0.000510,0.069971,0.000994,0.330517,...,0.471153,0.005779,0.503562,0.006668,0.548326,0.008155,0.656220,0.007668,0.556712,0.006890
7,7,31,hard_tiles,0.031709,0.000069,0.675892,0.000178,0.657393,0.001745,0.043098,...,0.462260,0.031059,0.521631,0.025054,0.553283,0.035261,0.655738,0.031584,0.558827,0.041484
8,8,33,tiled,0.505967,0.001476,0.000368,0.000009,0.047806,0.000661,0.530560,...,0.452595,0.009130,0.548252,0.016903,0.544844,0.009339,0.654214,0.001933,0.556039,0.005574
9,9,11,fine_concrete,0.898892,0.010863,0.199836,0.014416,0.249953,0.013632,0.909311,...,0.349727,0.032219,0.837058,0.015107,0.555237,0.030927,0.664717,0.022018,0.560071,0.029678


In [124]:
# uncomment and run cell to write file
df_groups_Xtrain_mean_std_norm.to_csv("../preprocessed-csv/groups_Xtrain_mean_std_normalized_0-1.csv", index=False)